# Artificial Intelligence Nanodegree

## Convolutional Neural Networks

## Project: Write an Algorithm for a Dog Identification App 

---

In this notebook, some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this project. You will not need to modify the included code beyond what is requested. Sections that begin with **'(IMPLEMENTATION)'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully! 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut.  Markdown cells can be edited by double-clicking the cell to enter edit mode.

The rubric contains _optional_ "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. If you decide to pursue the "Stand Out Suggestions", you should include the code in this IPython notebook.



---
### Why We're Here 

In this notebook, you will make the first steps towards developing an algorithm that could be used as part of a mobile or web app.  At the end of this project, your code will accept any user-supplied image as input.  If a dog is detected in the image, it will provide an estimate of the dog's breed.  If a human is detected, it will provide an estimate of the dog breed that is most resembling.  The image below displays potential sample output of your finished project (... but we expect that each student's algorithm will behave differently!). 

![Sample Dog Output](images/sample_dog_output.png)

In this real-world setting, you will need to piece together a series of models to perform different tasks; for instance, the algorithm that detects humans in an image will be different from the CNN that infers dog breed.  There are many points of possible failure, and no perfect algorithm exists.  Your imperfect solution will nonetheless create a fun user experience!

### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Import Datasets
* [Step 1](#step1): Detect Humans
* [Step 2](#step2): Detect Dogs
* [Step 3](#step3): Create a CNN to Classify Dog Breeds (from Scratch)
* [Step 4](#step4): Use a CNN to Classify Dog Breeds (using Transfer Learning)
* [Step 5](#step5): Create a CNN to Classify Dog Breeds (using Transfer Learning)
* [Step 6](#step6): Write your Algorithm
* [Step 7](#step7): Test Your Algorithm

---
<a id='step0'></a>
## Step 0: Import Datasets

### Import Dog Dataset

In the code cell below, we import a dataset of dog images.  We populate a few variables through the use of the `load_files` function from the scikit-learn library:
- `train_files`, `valid_files`, `test_files` - numpy arrays containing file paths to images
- `train_targets`, `valid_targets`, `test_targets` - numpy arrays containing onehot-encoded classification labels 
- `dog_names` - list of string-valued dog breed names for translating labels

In [ ]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

#### Output of this block
#-----
#There are 133 total dog categories. 
#There are 8351 total dog images.
#There are 6680 training dog images. 
#There are 835 validation dog images. 
#There are 836 test dog images.
#-----
####

### Import Human Dataset

In the code cell below, we import a dataset of human images, where the file paths are stored in the numpy array `human_files`.

In [ ]:
import random
random.seed(8675309)

# load filenames in shuffled human dataset
human_files = np.array(glob("lfw/*/*"))
random.shuffle(human_files)

# print statistics about the dataset
print('There are %d total human images.' % len(human_files))

#### Output of this block
#---
#There are 13233 total human images.
#---
#### 

---
<a id='step1'></a>
## Step 1: Detect Humans

We use OpenCV's implementation of [Haar feature-based cascade classifiers](http://docs.opencv.org/trunk/d7/d8b/tutorial_py_face_detection.html) to detect human faces in images.  OpenCV provides many pre-trained face detectors, stored as XML files on [github](https://github.com/opencv/opencv/tree/master/data/haarcascades).  We have downloaded one of these detectors and stored it in the `haarcascades` directory.

In the next code cell, we demonstrate how to use this detector to find human faces in a sample image.

In [ ]:
import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline                               

# extract pre-trained face detector
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

# load color (BGR) image
img = cv2.imread(human_files[3])
# convert BGR image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# find faces in image
faces = face_cascade.detectMultiScale(gray)

# print number of faces detected in the image
print('Number of faces detected:', len(faces))

# get bounding box for each detected face
for (x,y,w,h) in faces:
    # add bounding box to color image
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    
# convert BGR image to RGB for plotting
cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# display the image, along with bounding box
plt.imshow(cv_rgb)
plt.show()

#### Output of this block
#---
#print number of faces detected in the image - ('Number of faces detected:', 1)
#---
#### 

Before using any of the face detectors, it is standard procedure to convert the images to grayscale.  The `detectMultiScale` function executes the classifier stored in `face_cascade` and takes the grayscale image as a parameter.  

In the above code, `faces` is a numpy array of detected faces, where each row corresponds to a detected face.  Each detected face is a 1D array with four entries that specifies the bounding box of the detected face.  The first two entries in the array (extracted in the above code as `x` and `y`) specify the horizontal and vertical positions of the top left corner of the bounding box.  The last two entries in the array (extracted here as `w` and `h`) specify the width and height of the box.

### Write a Human Face Detector

We can use this procedure to write a function that returns `True` if a human face is detected in an image and `False` otherwise.  This function, aptly named `face_detector`, takes a string-valued file path to an image as input and appears in the code block below.

In [ ]:
# returns "True" if face is detected in image stored at img_path
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

### (IMPLEMENTATION) Assess the Human Face Detector

__Question 1:__ Use the code cell below to test the performance of the `face_detector` function.  
- What percentage of the first 100 images in `human_files` have a detected human face?  
- What percentage of the first 100 images in `dog_files` have a detected human face? 

Ideally, we would like 100% of human images with a detected face and 0% of dog images with a detected face.  You will see that our algorithm falls short of this goal, but still gives acceptable performance.  We extract the file paths for the first 100 images from each of the datasets and store them in the numpy arrays `human_files_short` and `dog_files_short`.

__Answer:__ 

----

98% of the humans were detected as humans!
11% of the dogs were detected as humans

----


In [ ]:
human_files_short = human_files[:100]
dog_files_short = train_files[:100]
# Do NOT modify the code above this line.

## TODO: Test the performance of the face_detector algorithm 
## on the images in human_files_short and dog_files_short.

human_count = 0
for expected_human_image in human_files_short:
    if face_detector(expected_human_image):
        human_count = human_count + 1
        
dog_count = 0
for expected_dog_image in dog_files_short:
    if face_detector(expected_dog_image):
        dog_count = dog_count + 1
print(str(human_count) + "% of the humans were detected as humans!")
print(str(dog_count) + "% of the dogs were detected as humans")

#### Output of this block 
#----
#98% of the humans were detected as humans!
#11% of the dogs were detected as humans
#----
#### 

__Question 2:__ This algorithmic choice necessitates that we communicate to the user that we accept human images only when they provide a clear view of a face (otherwise, we risk having unneccessarily frustrated users!). In your opinion, is this a reasonable expectation to pose on the user? If not, can you think of a way to detect humans in images that does not necessitate an image with a clearly presented face?

__Answer:__

On the app UI we can provide a virtual box and should ask user to ensure that face should be fit within the box. This will help in three ways: 

1. It provides users guidance or Soft standardisation (in terms of size) of the inputs which will help in optimising the CNNs to learn efficiently. 
2. Unconscious communication to user that app expects One complete face to work upon.
3. It helps in reducing the possibilities that a genuine user is thrown errors “Face is not captured properly” and resulting user frustuation and churn. 

Alternatively, we can use openCV libraries in real time to figure out if an image contains (http://docs.opencv.org/trunk/d7/d8b/tutorial_py_face_detection.html). 



We suggest the face detector from OpenCV as a potential way to detect human images in your algorithm, but you are free to explore other approaches, especially approaches that make use of deep learning :).  Please use the code cell below to design and test your own face detection algorithm.  If you decide to pursue this _optional_ task, report performance on each of the datasets.

In [ ]:
## (Optional) TODO: Report the performance of another  
## face detection algorithm on the LFW dataset
### Feel free to use as many code cells as needed.

---
<a id='step2'></a>
## Step 2: Detect Dogs

In this section, we use a pre-trained [ResNet-50](http://ethereon.github.io/netscope/#/gist/db945b393d40bfa26006) model to detect dogs in images.  Our first line of code downloads the ResNet-50 model, along with weights that have been trained on [ImageNet](http://www.image-net.org/), a very large, very popular dataset used for image classification and other vision tasks.  ImageNet contains over 10 million URLs, each linking to an image containing an object from one of [1000 categories](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).  Given an image, this pre-trained ResNet-50 model returns a prediction (derived from the available categories in ImageNet) for the object that is contained in the image.

In [ ]:
from keras.applications.resnet50 import ResNet50

# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

### Pre-process the Data

When using TensorFlow as backend, Keras CNNs require a 4D array (which we'll also refer to as a 4D tensor) as input, with shape

$$
(\text{nb_samples}, \text{rows}, \text{columns}, \text{channels}),
$$

where `nb_samples` corresponds to the total number of images (or samples), and `rows`, `columns`, and `channels` correspond to the number of rows, columns, and channels for each image, respectively.  

The `path_to_tensor` function below takes a string-valued file path to a color image as input and returns a 4D tensor suitable for supplying to a Keras CNN.  The function first loads the image and resizes it to a square image that is $224 \times 224$ pixels.  Next, the image is converted to an array, which is then resized to a 4D tensor.  In this case, since we are working with color images, each image has three channels.  Likewise, since we are processing a single image (or sample), the returned tensor will always have shape

$$
(1, 224, 224, 3).
$$

The `paths_to_tensor` function takes a numpy array of string-valued image paths as input and returns a 4D tensor with shape 

$$
(\text{nb_samples}, 224, 224, 3).
$$

Here, `nb_samples` is the number of samples, or number of images, in the supplied array of image paths.  It is best to think of `nb_samples` as the number of 3D tensors (where each 3D tensor corresponds to a different image) in your dataset!

In [ ]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

### Making Predictions with ResNet-50

Getting the 4D tensor ready for ResNet-50, and for any other pre-trained model in Keras, requires some additional processing.  First, the RGB image is converted to BGR by reordering the channels.  All pre-trained models have the additional normalization step that the mean pixel (expressed in RGB as $[103.939, 116.779, 123.68]$ and calculated from all pixels in all images in ImageNet) must be subtracted from every pixel in each image.  This is implemented in the imported function `preprocess_input`.  If you're curious, you can check the code for `preprocess_input` [here](https://github.com/fchollet/keras/blob/master/keras/applications/imagenet_utils.py).

Now that we have a way to format our image for supplying to ResNet-50, we are now ready to use the model to extract the predictions.  This is accomplished with the `predict` method, which returns an array whose $i$-th entry is the model's predicted probability that the image belongs to the $i$-th ImageNet category.  This is implemented in the `ResNet50_predict_labels` function below.

By taking the argmax of the predicted probability vector, we obtain an integer corresponding to the model's predicted object class, which we can identify with an object category through the use of this [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a). 

In [ ]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

### Write a Dog Detector

While looking at the [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a), you will notice that the categories corresponding to dogs appear in an uninterrupted sequence and correspond to dictionary keys 151-268, inclusive, to include all categories from `'Chihuahua'` to `'Mexican hairless'`.  Thus, in order to check to see if an image is predicted to contain a dog by the pre-trained ResNet-50 model, we need only check if the `ResNet50_predict_labels` function above returns a value between 151 and 268 (inclusive).

We use these ideas to complete the `dog_detector` function below, which returns `True` if a dog is detected in an image (and `False` if not).

In [ ]:
### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151)) 

### (IMPLEMENTATION) Assess the Dog Detector

__Question 3:__ Use the code cell below to test the performance of your `dog_detector` function.  
- What percentage of the images in `human_files_short` have a detected dog?  
- What percentage of the images in `dog_files_short` have a detected dog?

__Answer:__ 

No code execution output to be published from question 1 until now. 

-----
1% of the humans were detected as dogs
100% of the dogs were detected as dogs!

-----

In [ ]:
### TODO: Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short.
human_count = 0
for expected_human_image in human_files_short:
    if dog_detector(expected_human_image):
        human_count = human_count + 1
        
dog_count = 0
for expected_dog_image in dog_files_short:
    if dog_detector(expected_dog_image):
        dog_count = dog_count + 1
        
print(str(human_count) + "% of the humans were detected as dogs")
print(str(dog_count) + "% of the dogs were detected as dogs!")

#### Output of this block
#----
#1% of the humans were detected as dogs
#100% of the dogs were detected as dogs!
#----
####

---
<a id='step3'></a>
## Step 3: Create a CNN to Classify Dog Breeds (from Scratch)

Now that we have functions for detecting humans and dogs in images, we need a way to predict breed from images.  In this step, you will create a CNN that classifies dog breeds.  You must create your CNN _from scratch_ (so, you can't use transfer learning _yet_!), and you must attain a test accuracy of at least 1%.  In Step 5 of this notebook, you will have the opportunity to use transfer learning to create a CNN that attains greatly improved accuracy.

Be careful with adding too many trainable layers!  More parameters means longer training, which means you are more likely to need a GPU to accelerate the training process.  Thankfully, Keras provides a handy estimate of the time that each epoch is likely to take; you can extrapolate this estimate to figure out how long it will take for your algorithm to train. 

We mention that the task of assigning breed to dogs from images is considered exceptionally challenging.  To see why, consider that *even a human* would have great difficulty in distinguishing between a Brittany and a Welsh Springer Spaniel.  

Brittany | Welsh Springer Spaniel
- | - 
<img src="images/Brittany_02625.jpg" width="100"> | <img src="images/Welsh_springer_spaniel_08203.jpg" width="200">

It is not difficult to find other dog breed pairs with minimal inter-class variation (for instance, Curly-Coated Retrievers and American Water Spaniels).  

Curly-Coated Retriever | American Water Spaniel
- | -
<img src="images/Curly-coated_retriever_03896.jpg" width="200"> | <img src="images/American_water_spaniel_00648.jpg" width="200">


Likewise, recall that labradors come in yellow, chocolate, and black.  Your vision-based algorithm will have to conquer this high intra-class variation to determine how to classify all of these different shades as the same breed.  

Yellow Labrador | Chocolate Labrador | Black Labrador
- | -
<img src="images/Labrador_retriever_06457.jpg" width="150"> | <img src="images/Labrador_retriever_06455.jpg" width="240"> | <img src="images/Labrador_retriever_06449.jpg" width="220">

We also mention that random chance presents an exceptionally low bar: setting aside the fact that the classes are slightly imabalanced, a random guess will provide a correct answer roughly 1 in 133 times, which corresponds to an accuracy of less than 1%.  

Remember that the practice is far ahead of the theory in deep learning.  Experiment with many different architectures, and trust your intuition.  And, of course, have fun! 

### Pre-process the Data

We rescale the images by dividing every pixel in every image by 255.

In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

#### Output of this code block ####
# 100%|██████████| 6680/6680 [04:02<00:00, 27.55it/s]
# 100%|██████████| 835/835 [00:34<00:00, 24.23it/s]
# 100%|██████████| 836/836 [00:25<00:00, 32.56it/s]
####

### (IMPLEMENTATION) Model Architecture

Create a CNN to classify dog breed.  At the end of your code cell block, summarize the layers of your model by executing the line:
    
        model.summary()

We have imported some Python modules to get you started, but feel free to import as many modules as you need.  If you end up getting stuck, here's a hint that specifies a model that trains relatively fast on CPU and attains >1% test accuracy in 5 epochs:

![Sample CNN](images/sample_cnn.png)
           
__Question 4:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  If you chose to use the hinted architecture above, describe why you think that CNN architecture should work well for the image classification task.

__Answer:__ 

Why CNN architecture work well for images: 

Convolution layers capture the neighbour of an element. So near by pixels get togeter to explain overall features. In the 1st Convolution layers algorithm obtains the the low level features like edges etc. For the 2nd conv layer, the input is the feature map from the first layer. So each layer of the input is basically describing the locations of low level features in the original image. Now when you apply a set of filters on top of that (pass it through the 2nd conv layer), the output will be activations that represent higher level features. So, two edges can come together to represent an square etc. Other high level patters semicircles or triangles starts emerging. 

As you go through the network and go through more conv layers, you get activation maps that represent more and more complex features and an image eventually. For a colored image, CNN are easily scalable by breaking down images into 3-D vectors (from RGB). 


High Level Summary: 

The idea of having multiple layers of CNN was to extract the features from the images. To do this, the filters were halfed in each conv layer. There are max pooling layers between them to reduce the size. There were also dropout layers to prevent overfitting and ensure sufficient learning. A relu function was used in the dense layer, and then, for the output, a softmax since it's the best for classification.

Detailed Summary:

The Sequential model is used. it is basically linear stack of multiple layers. We can create a Sequential model by defining list of layer instances. 

Convolution layers:

The first layer in a CNN is always a Convolutional Layer. The model needs to be told expected size of input in first layer. Other layers in the model can infer size from the previous layers and other parameters (like striding etc.) Hence, a an input_shape tuple argument input_shape=((224, 224, 3)) is passed to the first layer. 
Convolution layer puts in filters (here we are using 2*2 filter - please see below section for explanaiton for filter size) to extract features from an image. Convolution layer (after multiplying with filter) deploys a activation functions to create a feature map.   

Pooling layers:
Pooling layers are used in tandem to convolution layers to to reduce variance and to reduce computation complexity. At the same time it and extract low level features from neighbourhood. 

3 set of Conolution and Pooling: 

First convolution layer determines the high level featues like edges and curves. To obtian high level features like eyes, body shape, nose we need to go deeper. However, if we go too deep information will become too dilute for network to train. Hence a depth of 3 is taken for the convolution. 

Dropout layers 
Dropout is a technique where randomly selected neurons are ignored randomly during training. This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass and any weight updates are not applied to the neuron on the backward pass. This is important to avoid overfitting and provide an opportunity to the lower weights nodes to contribute. 

Flatten layer
If we need to do classification, we would need to use dense layer as last layer. Fully connected layers can not take any (spatial or otherwise) into account for processing. Output of a CNN is a 3-D array preserving spatial information (nearby pixels preserves spatial informaiton). so we treat output of the last convolution layers as a large piece of unstructured data.

Dense layer
2 dense layers are used - with a dropout layer in between these. In a dense layer, every node in the layer is connected to every node in the preceding layer - hence fully connected. Size of Last dense layer is equal to the number of classes (133 dog breeds in this case). Size reduction (from flatter layer to 133) is done is two steps to preserve information. Last dense layer used softmax activation to help produce a probabilistic distribution among candidate clsses (breeds). 


Other key decisions for the parameters: 

1. Activation method: 

Rectified linear units ("ReLU") activation method is used for all, but the last layer. One major benefit of using "RELU" is the reduced likelihood of the gradient to vanish unlike in the sigmoid function where gradient reduces to very small number as absolute value of x increases. The other benefit of ReLUs is sparsity. Sparsity arises when a≤0. The more such units that exist in a layer the more sparse the resulting representation. Sigmoids on the other hand are always likely to dense representations. Sparse representations seem to be more scalable and beneficial than dense representations.

Last layer of a CNN model is generally a softmax activation layer. The ReLUs will not vanish the effect during your training process. However, when you want to deal with classification problems, they cannot help much. Simply speaking, the sigmoid function or ("ReLU") can only handle two classes, which is not what we expect (dog breeds are much more than two). 

2. Pooling method to use? pool size?
Max pooling extracts the most important features like edges whereas, average pooling extracts features so smoothly. For image data - I think max pooling is better for extracting the extreme features.

Pool size of 2*2 pooling reduces 75% data. A higher pooling size we run risk of loosing information. 

3. Nnumber of Conv2D layers
why 3 number?
Be careful with adding too many trainable layers! More parameters means longer training, which means you are more likely to need a GPU to accelerate the training process.

4. Kernal / filter size 
Its a trade off between speed and accuracy. For the given model, with filter size of 2*2 for all layers, model training time for each epoch was around 9-10 minutes (ETA: 550-560 seconds). This is not a large time that warrants a a negotiation for the accuracy - hence i decided to keep filter size at 2*2.

5. Drop out probability
In thi example with close to 22 million parameters in dense_1 layer, we can safely use higher drop out probabilities of 30%-50%.


 

Model summary: 

Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 223, 223, 16)  208         convolution2d_input_1[0][0]      

maxpooling2d_2 (MaxPooling2D)    (None, 111, 111, 16)  0           convolution2d_1[0][0]            

convolution2d_2 (Convolution2D)  (None, 110, 110, 32)  2080        maxpooling2d_2[0][0]             

maxpooling2d_3 (MaxPooling2D)    (None, 55, 55, 32)    0           convolution2d_2[0][0]            

convolution2d_3 (Convolution2D)  (None, 54, 54, 64)    8256        maxpooling2d_3[0][0]             

maxpooling2d_4 (MaxPooling2D)    (None, 27, 27, 64)    0           convolution2d_3[0][0]            

dropout_1 (Dropout)              (None, 27, 27, 64)    0           maxpooling2d_4[0][0]             

flatten_2 (Flatten)              (None, 46656)         0           dropout_1[0][0]                  

dense_1 (Dense)                  (None, 500)           23328500    flatten_2[0][0]                  

dropout_2 (Dropout)              (None, 500)           0           dense_1[0][0]                    

dense_2 (Dense)                  (None, 133)           66633       dropout_2[0][0]                  

Trainable params: 23,405,677
Non-trainable params: 0

References: 
http://web.engr.illinois.edu/~slazebni/spring14/lec24_cnn.pdf
https://adeshpande3.github.io/adeshpande3.github.io/The-9-Deep-Learning-Papers-You-Need-To-Know-About.html
https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks/
https://www.topbots.com/14-design-patterns-improve-convolutional-neural-network-cnn-architecture/
https://keras.io/getting-started/sequential-model-guide/
https://stats.stackexchange.com/questions/126238/what-are-the-advantages-of-relu-over-sigmoid-function-in-deep-neural-networks
https://www.quora.com/What-is-the-benefit-of-using-average-pooling-rather-than-max-pooling
https://stackoverflow.com/questions/41991337/optimal-dropout
https://arxiv.org/pdf/1606.02228v2.pdf
https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks/
https://www.tensorflow.org/tutorials/layers

In [ ]:
##TODO
# CNN architechture 
# Note to evaluatior - Following code has been done for Keras 1.2.2 verison. In keras verison 2.0+ 

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()
model.add(Conv2D(16,2,2,activation='relu', 
                        input_shape=((224, 224, 3))))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,2,2,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,2,2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(133, activation='softmax'))

model.summary()

### Compile the Model

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### (IMPLEMENTATION) Train the Model

Train your model in the code cell below.  Use model checkpointing to save the model that attains the best validation loss.

You are welcome to [augment the training data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), but this is not a requirement. 

In [ ]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 5

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          nb_epoch=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)


#### Output of training 
#-----
#Train on 6680 samples, validate on 835 samples
#Epoch 1/5
#6660/6680 [============================>.] - ETA: 1s - loss: 4.8793 - acc: 0.0177       Epoch 00000: val_loss improved from inf to 4.58487, saving model to saved_models/weights.best.from_scratch.hdf5
#6680/6680 [==============================] - 545s - loss: 4.8791 - acc: 0.0177 - val_loss: 4.5849 - val_acc: 0.0443
#Epoch 2/5
#6660/6680 [============================>.] - ETA: 1s - loss: 4.3611 - acc: 0.0595  Epoch 00001: val_loss improved from 4.58487 to 4.24696, saving model to saved_models/weights.best.from_scratch.hdf5
#6680/6680 [==============================] - 455s - loss: 4.3610 - acc: 0.0593 - val_loss: 4.2470 - val_acc: 0.0743
#Epoch 3/5
#6660/6680 [============================>.] - ETA: 1s - loss: 3.7529 - acc: 0.1514      Epoch 00002: val_loss improved from 4.24696 to 4.17603, saving model to saved_models/weights.best.from_scratch.hdf5
#6680/6680 [==============================] - 505s - loss: 3.7525 - acc: 0.1512 - val_loss: 4.1760 - val_acc: 0.1006
#Epoch 4/5
#6660/6680 [============================>.] - ETA: 1s - loss: 2.7235 - acc: 0.3520  Epoch 00003: val_loss did not improve
#6680/6680 [==============================] - 465s - loss: 2.7232 - acc: 0.3516 - val_loss: 4.5032 - val_acc: 0.0958
#Epoch 5/5
#6660/6680 [============================>.] - ETA: 1s - loss: 1.6008 - acc: 0.6024  Epoch 00004: val_loss did not improve
#6680/6680 [==============================] - 569s - loss: 1.6013 - acc: 0.6021 - val_loss: 5.3724 - val_acc: 0.0958
#-----
####


### Load the Model with the Best Validation Loss

In [ ]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

### Test the Model

Try out your model on the test dataset of dog images.  Ensure that your test accuracy is greater than 1%.

In [ ]:
#Test the model 
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

#### Output of this section
#---
# Test accuracy: 9.0000%
#
# Trained model accuracy is 9% which satisfies the condition that "Ensure that your test accuracy is greater than 1%"
#---
####

---
<a id='step4'></a>
## Step 4: Use a CNN to Classify Dog Breeds

To reduce training time without sacrificing accuracy, we show you how to train a CNN using transfer learning.  In the following step, you will get a chance to use transfer learning to train your own CNN.

### Obtain Bottleneck Features

In [ ]:
##Obtain Bottleneck features
bottleneck_features = np.load('bottleneck_features/DogVGG16Data.npz')
train_VGG16 = bottleneck_features['train']
valid_VGG16 = bottleneck_features['valid']
test_VGG16 = bottleneck_features['test']

### Model Architecture

The model uses the the pre-trained VGG-16 model as a fixed feature extractor, where the last convolutional output of VGG-16 is fed as input to our model.  We only add a global average pooling layer and a fully connected layer, where the latter contains one node for each dog category and is equipped with a softmax.

In [ ]:
VGG16_model = Sequential()
VGG16_model.add(GlobalAveragePooling2D(input_shape=train_VGG16.shape[1:]))
VGG16_model.add(Dense(133, activation='softmax'))

VGG16_model.summary()

#### Output of this section
#---
#____________________________________________________________________________________________________
#Layer (type)                     Output Shape          Param #     Connected to                     
#====================================================================================================
#globalaveragepooling2d_1 (Global (None, 512)           0           globalaveragepooling2d_input_1[0]
#____________________________________________________________________________________________________
#dense_3 (Dense)                  (None, 133)           68229       globalaveragepooling2d_1[0][0]   
#====================================================================================================
#Total params: 68,229
#Trainable params: 68,229
#Non-trainable params: 0
#____________________________________________________________________________________________________
#---
####

### Compile the Model

In [ ]:
VGG16_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

### Train the Model

In [ ]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.VGG16.hdf5', 
                               verbose=1, save_best_only=True)

VGG16_model.fit(train_VGG16, train_targets, 
          validation_data=(valid_VGG16, valid_targets),
          nb_epoch=20, batch_size=20, callbacks=[checkpointer], verbose=1)

#### Output of this section
#---
#Train on 6680 samples, validate on 835 samples
#Epoch 1/20
#6620/6680 [============================>.] - ETA: 0s - loss: 12.2270 - acc: 0.1189      Epoch 00000: val_loss improved from inf to 10.69091, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 5s - loss: 12.2193 - acc: 0.1193 - val_loss: 10.6909 - val_acc: 0.2120
#Epoch 2/20
#6620/6680 [============================>.] - ETA: 0s - loss: 10.0105 - acc: 0.2816Epoch 00001: val_loss improved from 10.69091 to 10.03765, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 2s - loss: 10.0231 - acc: 0.2813 - val_loss: 10.0376 - val_acc: 0.2814
#Epoch 3/20
#6500/6680 [============================>.] - ETA: 0s - loss: 9.5151 - acc: 0.3443Epoch 00002: val_loss improved from 10.03765 to 9.86755, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 2s - loss: 9.5019 - acc: 0.3446 - val_loss: 9.8675 - val_acc: 0.3018
#Epoch 4/20
#6620/6680 [============================>.] - ETA: 0s - loss: 9.2912 - acc: 0.3773Epoch 00003: val_loss improved from 9.86755 to 9.75346, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 2s - loss: 9.2992 - acc: 0.3769 - val_loss: 9.7535 - val_acc: 0.3138
#Epoch 5/20
#6500/6680 [============================>.] - ETA: 0s - loss: 9.1270 - acc: 0.3977Epoch 00004: val_loss improved from 9.75346 to 9.49013, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 2s - loss: 9.1283 - acc: 0.3973 - val_loss: 9.4901 - val_acc: 0.3246
#Epoch 6/20
#6540/6680 [============================>.] - ETA: 0s - loss: 8.8759 - acc: 0.4177 Epoch 00005: val_loss improved from 9.49013 to 9.30975, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 1s - loss: 8.8514 - acc: 0.4190 - val_loss: 9.3097 - val_acc: 0.3569
#Epoch 7/20
#6600/6680 [============================>.] - ETA: 0s - loss: 8.6612 - acc: 0.4355Epoch 00006: val_loss improved from 9.30975 to 9.18377, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 1s - loss: 8.6682 - acc: 0.4349 - val_loss: 9.1838 - val_acc: 0.3581
#Epoch 8/20
#6580/6680 [============================>.] - ETA: 0s - loss: 8.5462 - acc: 0.4500Epoch 00007: val_loss improved from 9.18377 to 9.06152, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 1s - loss: 8.5400 - acc: 0.4503 - val_loss: 9.0615 - val_acc: 0.3557
#Epoch 9/20
#6600/6680 [============================>.] - ETA: 0s - loss: 8.3994 - acc: 0.4623 Epoch 00008: val_loss improved from 9.06152 to 8.94871, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 1s - loss: 8.3789 - acc: 0.4635 - val_loss: 8.9487 - val_acc: 0.3737
#Epoch 10/20
#6560/6680 [============================>.] - ETA: 0s - loss: 8.2931 - acc: 0.4684Epoch 00009: val_loss improved from 8.94871 to 8.79203, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 1s - loss: 8.2760 - acc: 0.4695 - val_loss: 8.7920 - val_acc: 0.3880
#Epoch 11/20
#6520/6680 [============================>.] - ETA: 0s - loss: 8.1567 - acc: 0.4810Epoch 00010: val_loss improved from 8.79203 to 8.74466, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 2s - loss: 8.1549 - acc: 0.4813 - val_loss: 8.7447 - val_acc: 0.3832
#Epoch 12/20
#6600/6680 [============================>.] - ETA: 0s - loss: 8.0790 - acc: 0.4905 Epoch 00011: val_loss did not improve
#6680/6680 [==============================] - 1s - loss: 8.0986 - acc: 0.4891 - val_loss: 8.7766 - val_acc: 0.3820
#Epoch 13/20
#6660/6680 [============================>.] - ETA: 0s - loss: 8.0567 - acc: 0.4911Epoch 00012: val_loss improved from 8.74466 to 8.65963, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 1s - loss: 8.0567 - acc: 0.4912 - val_loss: 8.6596 - val_acc: 0.4012
#Epoch 14/20
#6500/6680 [============================>.] - ETA: 0s - loss: 7.9631 - acc: 0.4985Epoch 00013: val_loss improved from 8.65963 to 8.59466, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 1s - loss: 7.9809 - acc: 0.4973 - val_loss: 8.5947 - val_acc: 0.4132
#Epoch 15/20
#6600/6680 [============================>.] - ETA: 0s - loss: 7.9316 - acc: 0.4988Epoch 00014: val_loss did not improve
#6680/6680 [==============================] - 1s - loss: 7.9437 - acc: 0.4981 - val_loss: 8.5991 - val_acc: 0.3940
#Epoch 16/20
#6500/6680 [============================>.] - ETA: 0s - loss: 7.7399 - acc: 0.4986Epoch 00015: val_loss improved from 8.59466 to 8.30391, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 1s - loss: 7.7145 - acc: 0.5001 - val_loss: 8.3039 - val_acc: 0.4156
#Epoch 17/20
#6500/6680 [============================>.] - ETA: 0s - loss: 7.5967 - acc: 0.5186Epoch 00016: val_loss did not improve
#6680/6680 [==============================] - 1s - loss: 7.5998 - acc: 0.5187 - val_loss: 8.4394 - val_acc: 0.4132
#Epoch 18/20
#6480/6680 [============================>.] - ETA: 0s - loss: 7.5221 - acc: 0.5213Epoch 00017: val_loss improved from 8.30391 to 8.23424, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 1s - loss: 7.5320 - acc: 0.5202 - val_loss: 8.2342 - val_acc: 0.4180
#Epoch 19/20
#6660/6680 [============================>.] - ETA: 0s - loss: 7.3925 - acc: 0.5291Epoch 00018: val_loss improved from 8.23424 to 8.14524, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 1s - loss: 7.3993 - acc: 0.5287 - val_loss: 8.1452 - val_acc: 0.4204
#Epoch 20/20
#6540/6680 [============================>.] - ETA: 0s - loss: 7.3407 - acc: 0.5361 Epoch 00019: val_loss improved from 8.14524 to 8.13370, saving model to saved_models/weights.best.VGG16.hdf5
#6680/6680 [==============================] - 1s - loss: 7.3515 - acc: 0.5355 - val_loss: 8.1337 - val_acc: 0.4263
#---
####

### Load the Model with the Best Validation Loss

In [ ]:
VGG16_model.load_weights('saved_models/weights.best.VGG16.hdf5')

### Test the Model

Now, we can use the CNN to test how well it identifies breed within our test dataset of dog images.  We print the test accuracy below.

In [ ]:
# get index of predicted dog breed for each image in test set
VGG16_predictions = [np.argmax(VGG16_model.predict(np.expand_dims(feature, axis=0))) for feature in test_VGG16]

# report test accuracy
test_accuracy = 100*np.sum(np.array(VGG16_predictions)==np.argmax(test_targets, axis=1))/len(VGG16_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

#### Output of this section
#---
#Test accuracy: 43.0000%
#---
####

### Predict Dog Breed with the Model

In [ ]:
from extract_bottleneck_features import *

def VGG16_predict_breed(img_path):
    # extract bottleneck features
    bottleneck_feature = extract_VGG16(path_to_tensor(img_path))
    # obtain predicted vector
    predicted_vector = VGG16_model.predict(bottleneck_feature)
    # return dog breed that is predicted by the model
    return dog_names[np.argmax(predicted_vector)]

---
<a id='step5'></a>
## Step 5: Create a CNN to Classify Dog Breeds (using Transfer Learning)

You will now use transfer learning to create a CNN that can identify dog breed from images.  Your CNN must attain at least 60% accuracy on the test set.

In Step 4, we used transfer learning to create a CNN using VGG-16 bottleneck features.  In this section, you must use the bottleneck features from a different pre-trained model.  To make things easier for you, we have pre-computed the features for all of the networks that are currently available in Keras:
- [VGG-19](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogVGG19Data.npz) bottleneck features
- [ResNet-50](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogResnet50Data.npz) bottleneck features
- [Inception](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogInceptionV3Data.npz) bottleneck features
- [Xception](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogXceptionData.npz) bottleneck features

The files are encoded as such:

    Dog{network}Data.npz
    
where `{network}`, in the above filename, can be one of `VGG19`, `Resnet50`, `InceptionV3`, or `Xception`.  Pick one of the above architectures, download the corresponding bottleneck features, and store the downloaded file in the `bottleneck_features/` folder in the repository.

### (IMPLEMENTATION) Obtain Bottleneck Features

In the code block below, extract the bottleneck features corresponding to the train, test, and validation sets by running the following:

    bottleneck_features = np.load('bottleneck_features/Dog{network}Data.npz')
    train_{network} = bottleneck_features['train']
    valid_{network} = bottleneck_features['valid']
    test_{network} = bottleneck_features['test']

In [ ]:
### TODO: Obtain bottleneck features from another pre-trained CNN.
bottleneck_features = np.load('bottleneck_features/DogResnet50Data.npz')
train_DogResnet50 = bottleneck_features['train']
valid_DogResnet50 = bottleneck_features['valid']
test_DogResnet50 = bottleneck_features['test']

### (IMPLEMENTATION) Model Architecture

Create a CNN to classify dog breed.  At the end of your code cell block, summarize the layers of your model by executing the line:
    
        <your model's name>.summary()
   
__Question 5:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  Describe why you think the architecture is suitable for the current problem.

__Answer:__ 

Deep neural networks has a huge number of parameters, often in the range of millions (22 millions in our build from scratch model). Training a CNN on a small dataset (We had only 6680 training images for dogs as compared to 22 million parameters) greatly affects the Covnet’s ability to generalize. 

Alternatively we can fine-tune existing networks (Resnet 50 in this case) that are trained on a large dataset (millions of RGB type general images) by continue training it (i.e. running back-propagation) on the smaller dataset we have. Provided that our dataset is not drastically different in context to the original dataset (e.g. Resnet 50), the pre-trained model will already have learned features that are relevant to our own classification problem.

We will only keep the convolutional part of the model, everything up to the fully-connected layers. Then we will train a small fully-connected model (using softmax activation function) on top of the stored features. I have used pooling layer to reduce the complexity.


References: 
https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
https://flyyufelix.github.io/2016/10/03/fine-tuning-in-keras-part1.html
https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html


Model Summary: 

Layer (type)                     Output Shape          Param #     Connected to                     

globalaveragepooling2d_2 (Global (None, 2048)          0           globalaveragepooling2d_input_2[0]

dense_4 (Dense)                  (None, 133)           272517      globalaveragepooling2d_2[0][0]   

Total params: 272,517
Trainable params: 272,517
Non-trainable params: 0
____________________________________________________________________________________________________





In [ ]:
### TODO: Define your architecture.
Resnet50_model = Sequential()
Resnet50_model.add(GlobalAveragePooling2D(input_shape=train_DogResnet50.shape[1:]))
Resnet50_model.add(Dense(133, activation='softmax'))

Resnet50_model.summary()

#### Output of this section
#---
#____________________________________________________________________________________________________
#Layer (type)                     Output Shape          Param #     Connected to                     
#====================================================================================================
#globalaveragepooling2d_2 (Global (None, 2048)          0           globalaveragepooling2d_input_2[0]
#____________________________________________________________________________________________________
#dense_4 (Dense)                  (None, 133)           272517      globalaveragepooling2d_2[0][0]   
#====================================================================================================
#Total params: 272,517
#Trainable params: 272,517
#Non-trainable params: 0
#____________________________________________________________________________________________________
#---
####

### (IMPLEMENTATION) Compile the Model

In [ ]:
### TODO: Compile the model.
Resnet50_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

### (IMPLEMENTATION) Train the Model

Train your model in the code cell below.  Use model checkpointing to save the model that attains the best validation loss.  

You are welcome to [augment the training data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), but this is not a requirement. 

In [ ]:
### TODO: Train the model.
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.ResNet50.hdf5', 
                               verbose=1, save_best_only=True)

Resnet50_model.fit(train_DogResnet50, train_targets, 
          validation_data=(valid_DogResnet50, valid_targets),
          nb_epoch=20, batch_size=20, callbacks=[checkpointer], verbose=1)

#### Output of this section
#---
#Train on 6680 samples, validate on 835 samples
#Epoch 1/20
#6560/6680 [============================>.] - ETA: 0s - loss: 1.6080 - acc: 0.6047      Epoch 00000: val_loss improved from inf to 0.78196, saving model to saved_models/weights.best.ResNet50.hdf5
#6680/6680 [==============================] - 3s - loss: 1.5942 - acc: 0.6069 - val_loss: 0.7820 - val_acc: 0.7581
#Epoch 2/20
#6660/6680 [============================>.] - ETA: 0s - loss: 0.4357 - acc: 0.8689Epoch 00001: val_loss improved from 0.78196 to 0.62034, saving model to saved_models/weights.best.ResNet50.hdf5
#6680/6680 [==============================] - 2s - loss: 0.4350 - acc: 0.8692 - val_loss: 0.6203 - val_acc: 0.8000
#Epoch 3/20
#6660/6680 [============================>.] - ETA: 0s - loss: 0.2595 - acc: 0.9186Epoch 00002: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.2594 - acc: 0.9186 - val_loss: 0.7089 - val_acc: 0.7940
#Epoch 4/20
#6560/6680 [============================>.] - ETA: 0s - loss: 0.1784 - acc: 0.9466Epoch 00003: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.1782 - acc: 0.9469 - val_loss: 0.6957 - val_acc: 0.7940
#Epoch 5/20
#6600/6680 [============================>.] - ETA: 0s - loss: 0.1211 - acc: 0.9620Epoch 00004: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.1234 - acc: 0.9615 - val_loss: 0.7005 - val_acc: 0.7868
#Epoch 6/20
#6660/6680 [============================>.] - ETA: 0s - loss: 0.0932 - acc: 0.9712Epoch 00005: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0933 - acc: 0.9711 - val_loss: 0.7036 - val_acc: 0.8048
#Epoch 7/20
#6620/6680 [============================>.] - ETA: 0s - loss: 0.0634 - acc: 0.9808Epoch 00006: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0636 - acc: 0.9804 - val_loss: 0.7017 - val_acc: 0.8072
#Epoch 8/20
#6600/6680 [============================>.] - ETA: 0s - loss: 0.0488 - acc: 0.9864Epoch 00007: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0498 - acc: 0.9861 - val_loss: 0.6858 - val_acc: 0.8240
#Epoch 9/20
#6600/6680 [============================>.] - ETA: 0s - loss: 0.0364 - acc: 0.9897Epoch 00008: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0369 - acc: 0.9895 - val_loss: 0.7133 - val_acc: 0.8084
#Epoch 10/20
#6560/6680 [============================>.] - ETA: 0s - loss: 0.0286 - acc: 0.9931Epoch 00009: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0288 - acc: 0.9930 - val_loss: 0.7451 - val_acc: 0.8132
#Epoch 11/20
#6620/6680 [============================>.] - ETA: 0s - loss: 0.0209 - acc: 0.9952Epoch 00010: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0216 - acc: 0.9949 - val_loss: 0.7429 - val_acc: 0.8228
#Epoch 12/20
#6620/6680 [============================>.] - ETA: 0s - loss: 0.0189 - acc: 0.9953Epoch 00011: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0187 - acc: 0.9954 - val_loss: 0.8350 - val_acc: 0.8012
#Epoch 13/20
#6520/6680 [============================>.] - ETA: 0s - loss: 0.0164 - acc: 0.9960Epoch 00012: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0161 - acc: 0.9961 - val_loss: 0.8234 - val_acc: 0.8120
#Epoch 14/20
#6580/6680 [============================>.] - ETA: 0s - loss: 0.0128 - acc: 0.9970Epoch 00013: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0129 - acc: 0.9969 - val_loss: 0.8186 - val_acc: 0.8323
#Epoch 15/20
#6660/6680 [============================>.] - ETA: 0s - loss: 0.0117 - acc: 0.9971Epoch 00014: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0117 - acc: 0.9972 - val_loss: 0.8579 - val_acc: 0.8240
#Epoch 16/20
#6660/6680 [============================>.] - ETA: 0s - loss: 0.0084 - acc: 0.9980    Epoch 00015: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0084 - acc: 0.9981 - val_loss: 0.8399 - val_acc: 0.8335
#Epoch 17/20
#6660/6680 [============================>.] - ETA: 0s - loss: 0.0078 - acc: 0.9980    Epoch 00016: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0077 - acc: 0.9981 - val_loss: 0.8504 - val_acc: 0.8275
#Epoch 18/20
#6620/6680 [============================>.] - ETA: 0s - loss: 0.0072 - acc: 0.9980    Epoch 00017: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0072 - acc: 0.9981 - val_loss: 0.8493 - val_acc: 0.8263
#Epoch 19/20
#6520/6680 [============================>.] - ETA: 0s - loss: 0.0048 - acc: 0.9989    Epoch 00018: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0068 - acc: 0.9987 - val_loss: 0.8923 - val_acc: 0.8192
#Epoch 20/20
#6640/6680 [============================>.] - ETA: 0s - loss: 0.0061 - acc: 0.9983    Epoch 00019: val_loss did not improve
#6680/6680 [==============================] - 2s - loss: 0.0061 - acc: 0.9984 - val_loss: 0.9085 - val_acc: 0.8287
#---
####

### (IMPLEMENTATION) Load the Model with the Best Validation Loss

In [ ]:
### TODO: Load the model weights with the best validation loss.
Resnet50_model.load_weights('saved_models/weights.best.ResNet50.hdf5')

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images. Ensure that your test accuracy is greater than 60%.

In [ ]:
### Test the model 
### TODO: Calculate classification accuracy on the test dataset.
Resnet50_predictions = [np.argmax(Resnet50_model.predict(np.expand_dims(feature, axis=0))) for feature in test_DogResnet50]

# report test accuracy
test_accuracy = 100*np.sum(np.array(Resnet50_predictions)==np.argmax(test_targets, axis=1))/len(Resnet50_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

#### Output of this section
#---
# Test accuracy: 79.0000%
# Test accuracy is 79% which satisfies the criteria that "Ensure that your test accuracy is greater than 60%."
#---
####

### (IMPLEMENTATION) Predict Dog Breed with the Model

Write a function that takes an image path as input and returns the dog breed (`Affenpinscher`, `Afghan_hound`, etc) that is predicted by your model.  

Similar to the analogous function in Step 5, your function should have three steps:
1. Extract the bottleneck features corresponding to the chosen CNN model.
2. Supply the bottleneck features as input to the model to return the predicted vector.  Note that the argmax of this prediction vector gives the index of the predicted dog breed.
3. Use the `dog_names` array defined in Step 0 of this notebook to return the corresponding breed.

The functions to extract the bottleneck features can be found in `extract_bottleneck_features.py`, and they have been imported in an earlier code cell.  To obtain the bottleneck features corresponding to your chosen CNN architecture, you need to use the function

    extract_{network}
    
where `{network}`, in the above filename, should be one of `VGG19`, `Resnet50`, `InceptionV3`, or `Xception`.

In [ ]:
### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.
def dog_breed(img_path):
    # extract bottleneck features
    bottleneck_feature = extract_Resnet50(path_to_tensor(img_path))
    
    # obtain predicted vector
    predicted_vector = Resnet50_model.predict(bottleneck_feature)
    
    # return dog breed that is predicted by the model
    return dog_names[np.argmax(predicted_vector)]

---
<a id='step6'></a>
## Step 6: Write your Algorithm

Write an algorithm that accepts a file path to an image and first determines whether the image contains a human, dog, or neither.  Then,
- if a __dog__ is detected in the image, return the predicted breed.
- if a __human__ is detected in the image, return the resembling dog breed.
- if __neither__ is detected in the image, provide output that indicates an error.

You are welcome to write your own functions for detecting humans and dogs in images, but feel free to use the `face_detector` and `dog_detector` functions developed above.  You are __required__ to use your CNN from Step 5 to predict dog breed.  

Some sample output for our algorithm is provided below, but feel free to design your own user experience!

![Sample Human Output](images/sample_human_output.png)


### (IMPLEMENTATION) Write your Algorithm

In [ ]:
### TODO: Write your algorithm.
### Feel free to use as many code cells as needed.

def dog_breed_detector(img_path):
    breed = dog_breed(img_path) 
    
    # Display the image
    img = cv2.imread(img_path)
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(cv_rgb)
    plt.show()
    
    # Detect what it is
    if face_detector(img_path):
        print("That's a human, but it looks like a " + str(breed))
    elif dog_detector(img_path):
        print("That's a dog. Breed: " + str(breed))
    else:
        print("I can't detect anything!")
        

---
<a id='step7'></a>
## Step 7: Test Your Algorithm

In this section, you will take your new algorithm for a spin!  What kind of dog does the algorithm think that __you__ look like?  If you have a dog, does it predict your dog's breed accurately?  If you have a cat, does it mistakenly think that your cat is a dog?

### (IMPLEMENTATION) Test Your Algorithm on Sample Images!

Test your algorithm at least six images on your computer.  Feel free to use any images you like.  Use at least two human and two dog images.  

__Question 6:__ Is the output better than you expected :) ?  Or worse :( ?  Provide at least three possible points of improvement for your algorithm.

__Answer:__ 

Output: 

Image 1: "That's a dog. Breed: Silky_terrier"
Right. Actually 3 dogs are present in the picture. Out of which 2 are silky terrier and one is austrialian terrier - which is very close to the silky terrier in looks. Algorithm identifies correctly that a dog is present in the picture. So, correct prediction.

Image 2: "That's a dog. Breed: Silky_terrier". 
Wrong. Identifies correctly that a dog is present in the picture. Actual breed of dog is Austrailian terrier. So, wrong  prediction. But to be fair to algorithm, Silky_terrier and austrilian terrier are quite similar to confuse even human eyes. 

Image 3: "That's a dog. Breed: Dachshund"
Wrong. It is actually a picture of european politician. Algorithm inaccurately does not recodnize human but instead wrongly predicts a dog. 

Image 4: "That's a dog. Breed: Field_spaniel"
Right. Identifies correctly that a dog is present in the picture. Actual breed of dog is Field spaniel. So, correct prediction.

Image 5: "I can't detect anything!" 
Wrong. It contains picture for both human and dog (both clear), algorithm could not find neither dog nor human. This was a set-back. as in algorithm we are checking for human first, I was expecting to pick up human face atleast. 

Image 6: "That's a dog. Breed: Akita". 
Right. Identifies correctly that a dog is present in the picture. Actual breed of dog is akita. So, correct prediction. 

Image 7: "That's a human, but it looks like a Black_russian_terrier".
Right. It is actually a picture of hollywood celibrity. Algorithm identifies correctly that no dog is in the picture. Only one human face was present in the picture. Prediction of "Black_russian_terrier" may be driven by fact that human in the picture was wearing a balck suit. 


Reference: http://dogtime.com/dog-breeds/profiles

Output is actually better than i expected. Out of 8 cases: 
- 1 case was a true setback where algorithm wrongly identifies a human face as a dog. 
- 1 case where there was a dog and human together - algo is not trained to handle such cases (we trained it only with clear images of one dog or one human). 
- 1 case case where algorithm was able to predict with decent accuracy a group of dogs - this was not expected actually. 
- Other cases algo predicts human and dog correctly - with correct breed also, in some cases a close enough breed. 

Three possible improvements in our algorithm: 

1. Model gets confused between two similar looking breeds (to be fair, so does me). Having a larger training and a deeper network set might help. 
2. We could have augmented training set by having rotated versions of training pictures. 
3. Running time (or response time) of the algorithm is not upto the mark to be used in profesisonal apps. Could we have some parallel network learning to reduce the response time ?

￼

In [ ]:
## Load the cell
sample_files = np.array(glob("sample_pictures_step7/*"))
print(sample_files)

## testing for sample pictures 
for path in sample_files:
    dog_breed_detector(path)
    
#### Output of this section
#---
#['sample_pictures_step7/1. multiple terriers .jpeg'
# 'sample_pictures_step7/Austrailian terrier.jpg'
# 'sample_pictures_step7/download (3).jpeg'
# 'sample_pictures_step7/field-spaniel-dogs-puppies-3.jpg'
# 'sample_pictures_step7/test_1.jpeg' 'sample_pictures_step7/test_2.jpeg'
# 'sample_pictures_step7/test_3.jpeg' 'sample_pictures_step7/test_4.jpeg'
# 'sample_pictures_step7/test_5.jpeg' 'sample_pictures_step7/test_6.png']
#
# <Image 1>
# That's a dog. Breed: Silky_terrier
#
# <Image 2>
# That's a dog. Breed: Silky_terrier
#
# <Image 3>
# That's a dog. Breed: Dachshund
#
# <Image 4>
# That's a dog. Breed: Field_spaniel
#
# <Image 5>
# I can't detect anything!
#
# <Image 6>
# That's a dog. Breed: Akita
#
# <Image 7>
# That's a human, but it looks like a Black_russian_terrier
#
# 
#---
####